In [1]:
import os
from datetime import datetime, timedelta  # For working with dates

import pandas as pd  # For working with DataFrames
from dotenv import load_dotenv
from sqlalchemy import create_engine  # For creating a connection engine

from reports.formatter import formatter

##################### LOADING IMPORTANT DATA ######################
# Load environment variables from the .env file
env_file_path = 'D:/Projects/.env'
load_dotenv(env_file_path)
# Giving output file name
output_file_path = 'HOURLY.xlsx'
# Load data from different sheets in 'promotion.xlsx' into DataFrames
promotion_path = r'D:\Projects\promotion.xlsx'
region_df = pd.read_excel(promotion_path, sheet_name='Region')
aksiya_df = pd.read_excel(promotion_path, sheet_name='Aksiya')
paket_df = pd.read_excel(promotion_path, sheet_name='Paket')
types_df = pd.read_excel(promotion_path, sheet_name='TYPES')

##################### ACCESS ENV VARIABLES ######################
db_server = os.getenv("DB_SERVER")
db_database = os.getenv("DB_DATABASE_SERGELI")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_port = os.getenv("DB_PORT")
db_driver_name = os.getenv("DB_DRIVER_NAME")

##################### PROCEDURE NAME ######################
procedure_name = os.getenv("HOURLY_SHORT")  # THIS IS HOURLY DATA GATHERING

##################### DATE - MARCH ######################
CURRENT_MONTH = datetime.now().month
CURRENT_YEAR = datetime.now().year
today_date = datetime.now().strftime('%Y%m%d')
tomorrow_date = (datetime.now() + timedelta(days=1)).strftime('%Y%m%d')

##################### CONNECTION STRING AND SQL QUERY ######################
# Construct the connection string
conn_str = f"mssql+pyodbc://{db_user}:{db_password}@{db_server}:{db_port}/{db_database}?driver={db_driver_name}"
engine = create_engine(conn_str)

sql_query = f"""
    DECLARE @DateBegin DATE = ?;
    DECLARE @DateEnd DATE = ?;

    EXEC {procedure_name}
        @DateBegin = @DateBegin,
        @DateEnd = @DateBegin;
"""

#####################  EXECUTION  ######################
df = pd.read_sql_query(sql_query, engine, params=(today_date, tomorrow_date))

##################### BASIC FILTER ######################
df['DataEntered'] = pd.to_datetime(df['DataEntered'])
df.columns = ['DocKind', 'InvoiceNumber', 'Goodid', 'GoodName', 'Manufacturer', 'INN', 'ClientName',
              'InvoiceManager',
              'ClientMan', 'PaymentTerm',
              'BasePrice', 'SellingPrice', 'Quantity', 'DataEntered', 'BaseAmount', 'TotalAmount']

df = df[(df['DataEntered'].dt.month == CURRENT_MONTH)
        & (df['DataEntered'].dt.year == CURRENT_YEAR)
        & (df['DataEntered'].dt.date == datetime.now().date())
        & df['DocKind'].isin(['Оптовая реализация', 'Финансовая скидка'])]

region_df['ClientMan'] = region_df['ClientMan'].str.title()
df['ClientMan'] = df['ClientMan'].str.title()

df = pd.merge(df, aksiya_df[['Goodid', 'Aksiya']], left_on='Goodid', right_on='Goodid', how='left')
df = pd.merge(df, paket_df[['Goodid', 'Paket']], left_on='Goodid', right_on='Goodid', how='left')
df = pd.merge(df, region_df[['ClientMan', 'Region']], left_on='ClientMan', right_on='ClientMan', how='left')

df['OXVAT'] = df['INN'].map(df['INN'].value_counts())
#
df['inn_temp'] = pd.to_numeric(df['INN'], errors='coerce')
types_df['INN_temp'] = pd.to_numeric(types_df['INN'], errors='coerce')
df = pd.merge(df, types_df[['INN_temp', 'TYPE', 'RegionType']], left_on='inn_temp', right_on='INN_temp', how='left')
#
# ##################### ROZ | SET | OPT  ######################
df['TYPE'] = df['TYPE'].fillna('ROZ')
df.loc[df['TYPE'] == 'ROZ', 'RegionType'] = df['Region']
df.drop(['INN_temp', 'inn_temp'], axis=1, inplace=True)

##################### CONVERT TO CATEGORICAL DATA TYPE ######################
df.sort_values(by='DataEntered', ascending=False, inplace=True)
df.to_excel(output_file_path, index=False)

##################### FORMAT THE TABLE  ######################
formatter(df, output_file_path)
##################### MODIFIED TIME OF THE FILE ######################

if os.path.exists(output_file_path):
    # Get the size of the file in bytes
    file_size_bytes = os.path.getsize(output_file_path)

    # Convert bytes to kilobytes, megabytes, or gigabytes for readability
    file_size_kb = file_size_bytes / 1024.0
    file_size_mb = file_size_kb / 1024.0
    file_size_gb = file_size_mb / 1024.0

    print(
        f"File Size: {file_size_bytes} bytes, {file_size_kb:.2f} KB, {file_size_mb:.2f} MB, {file_size_gb:.2f} GB")

    # Get the last modification time in seconds since the epoch
    modification_time_seconds = os.path.getmtime(output_file_path)

    # Convert seconds since the epoch to a datetime object
    modification_time = datetime.fromtimestamp(modification_time_seconds)

    # Get the change time (metadata change time) in seconds since the epoch
    change_time_seconds = os.path.getctime(output_file_path)

    # Convert seconds since the epoch to a datetime object
    change_time = datetime.fromtimestamp(change_time_seconds)

    print(f"Last Modified Time: {modification_time.strftime('%H:%M')}")
    print(f"Change Time: {change_time.strftime('%H:%M')}")

else:
    print("File not found.")

Formatting🎨....
Formatting took: 0.03 seconds.

File Size: 8259 bytes, 8.07 KB, 0.01 MB, 0.00 GB
Last Modified Time: 18:19
Change Time: 15:56


# FORMATTING

In [6]:
def calculate_oxvat(filtered_d, region):
    """
    Calculate the count of unique clients based on the provided conditions.
    
    Returns:
    int: The count of unique clients for the given region.
    """
    return len(filtered_d[(filtered_d['RegionType'] == region) & (filtered_d['TYPE'] == 'ROZ')]['INN'].unique())


def calculate_fact_to(region, region_type, *aksiya_value):
    """
    Calculate the 'Факт ТО' for a specific region, product type, and aksiya value.
    Returns:
    float: The calculated 'Факт ТО'.
    """
    filtered_region_df = filtered_df[(filtered_df['Region'] == region) & (filtered_df['TYPE'] == region_type)]

    if aksiya_value:
        filtered_region_df = filtered_region_df[filtered_region_df['Aksiya'] == aksiya_value[0]]
        return max(filtered_region_df['Quantity'].sum(), 0)

    return max(filtered_region_df['TotalAmount'].sum(), 0)


def calculate_percentage(fact_column, plan_column):
    """
    Calculate the percentage based on 'Факт' and 'План' columns.

    Returns:
    Series: The calculated percentage column.
    """
    return (fact_column / plan_column).fillna(1) * 100


# Load data from hourly.xlsx
input_file_path = 'HOURLY.xlsx'
plan_path = r'D:/Projects/plan.xlsx'
# Read plan data from plan.xlsx
plan_df = pd.read_excel(plan_path, sheet_name="ROZ")
df = pd.read_excel(input_file_path)

# Filter the DataFrame to include only rows where 'TYPE' is 'ROZ'
filtered_df = df[df['TYPE'] == 'ROZ']

# Define unique values for TYPE
unique_types = filtered_df['TYPE'].unique()

# Defined the specific Plan Columns i need
region_combinations = plan_df[['Region', 'TO', 'Риноксил', 'Альцетро', 'Forsil', 'Тризим Таб', 'Тризим Кап', 'Энтеросгель']]

# Calculate 'Факт ТО' and '%' for each product type
for product_type in unique_types:
    fact_column = f'Fact ТО {product_type}'
    percent_column = f'% {product_type}'
    
    region_combinations = region_combinations.assign(**{fact_column: region_combinations['Region'].apply(
    lambda region: calculate_fact_to(region, product_type))})

    
    region_combinations.loc[:, percent_column] = calculate_percentage(region_combinations[fact_column],
                                                                      region_combinations['TO'])
    

# Add 'OXVAT' column based on the provided formula
region_combinations['OXVAT'] = region_combinations['Region'].apply(lambda region: calculate_oxvat(filtered_df, region))

# Альцетро
region_combinations['Fact Альцетро'] = region_combinations['Region'].apply(
    lambda region: calculate_fact_to(region, 'ROZ', 'Альцетро'))
region_combinations['% Альцетро'] = calculate_percentage(region_combinations['Fact Альцетро'],
                                                         region_combinations['Альцетро'])

# FORSIL
region_combinations['Fact FORSIL'] = region_combinations['Region'].apply(
    lambda region: calculate_fact_to(region, 'ROZ', 'Форсил'))
region_combinations['% FORSIL'] = calculate_percentage(region_combinations['Fact FORSIL'],
                                                       region_combinations['Forsil'])
# Тризим Таб
region_combinations['Fact Тризим Таб'] = region_combinations['Region'].apply(
    lambda region: calculate_fact_to(region, 'ROZ', 'Тризим №20'))
region_combinations['% Тризим Таб'] = calculate_percentage(region_combinations['Fact Тризим Таб'],
                                                           region_combinations['Тризим Таб'])

# Риноксил
region_combinations['Fact Риноксил'] = region_combinations['Region'].apply(
    lambda region: calculate_fact_to(region, 'ROZ', 'Риноксил'))
region_combinations['% Риноксил'] = calculate_percentage(region_combinations['Fact Риноксил'],
                                                         region_combinations['Риноксил'])

# Риномакс Таб
region_combinations['Fact Риномакс Таб'] = region_combinations['Region'].apply(
    lambda region: calculate_fact_to(region, 'ROZ', 'Риномакс Таб'))
region_combinations['% Риномакс Таб'] = calculate_percentage(region_combinations['Fact Риномакс Таб'],
                                                             region_combinations['Риномакс Таб'])

# TRIZIM KAP
region_combinations['Fact Тризим Кап'] = region_combinations['Region'].apply(
    lambda region: calculate_fact_to(region, 'ROZ', 'Тризим Кап'))
region_combinations['% Тризим Кап'] = calculate_percentage(region_combinations['Fact Тризим Кап'],
                                                           region_combinations['Тризим Кап'])
# Энтеросгель
region_combinations['Fact Энтеросгель'] = region_combinations['Region'].apply(
    lambda region: calculate_fact_to(region, 'ROZ', 'Энтеросгель'))
region_combinations['% Энтеросгель'] = calculate_percentage(region_combinations['Fact Энтеросгель'],
                                                            region_combinations['Энтеросгель'])
# COLUMN RENAME TO RUSSIAN LETTERS FROM ENGLISH
column_mapping = {
    'Region': 'Регион',
    'TO': 'TO',
    'Fact ТО ROZ': 'Факт ТО',
    '% ROZ': '% TO',
    'OXVAT': 'Охват',

    'Альцетро': 'Альцетро',
    'Fact Альцетро': 'Альцетро Альцетро',
    '% Альцетро': '% Альцетро',
    
    'Риноксил': 'Риноксил',
    'Fact Риноксил': 'Факт Риноксил',
    '% Риноксил': '% Риноксил',

    'Риномакс Таб': 'Риномакс Таб',
    'Fact Риномакс Таб': 'Факт Риномакс Таб',
    '% Риномакс Таб': '% Риномакс Таб',

    'Forsil': 'Форсил',
    'Fact FORSIL': 'Факт Форсил',
    '% FORSIL': '% Форсил',

    'Тризим Таб': 'Тризим Таб',
    'Fact Тризим Таб': 'Факт Тризим Таб',
    '% Тризим Таб': '% Тризим Таб',

    'Тризим Кап': 'Тризим Кап',
    'Fact Тризим Кап': 'Факт Тризим Кап',
    '% Тризим Кап': '% Тризим Кап',

    'Энтеросгель': 'Энтеросгель',
    'Fact Энтеросгель': 'Факт Энтеросгель',
    '% Энтеросгель': '% Энтеросгель'
}

# Rename the columns using the mapping dictionary
region_combinations = region_combinations.rename(columns=column_mapping)
# Sorting to look it pretty
region_combinations.sort_values(by='% TO', ascending=False, inplace=True)

KeyError: 'Риномакс Таб'

In [ ]:
total_row = pd.Series({
    'Регион': 'TOTAL',
    'TO': region_combinations['TO'].sum(),
    'Факт ТО': region_combinations['Факт ТО'].sum(),
    '% TO': (region_combinations['Факт ТО'].sum() / region_combinations['TO'].sum()) * 100,
    'Охват': region_combinations['Охват'].sum(),
    'Риномакс': region_combinations['Риномакс'].sum(),
    'Факт Риномакс': region_combinations['Факт Риномакс'].sum(),
    '% Риномакс': (region_combinations['Факт Риномакс'].sum() / region_combinations['Риномакс'].sum()) * 100,
    'Форсил': region_combinations['Форсил'].sum(),
    'Факт Форсил': region_combinations['Факт Форсил'].sum(),
    '% Форсил': (region_combinations['Факт Форсил'].sum() / region_combinations['Форсил'].sum()) * 100,
    'Тризим Таб': region_combinations['Тризим Таб'].sum(),
    'Факт Тризим Таб': region_combinations['Факт Тризим Таб'].sum(),
    '% Тризим Таб': (region_combinations['Факт Тризим Таб'].sum() / region_combinations['Тризим Таб'].sum()) * 100,
    'Тризим Кап': region_combinations['Тризим Кап'].sum(),
    'Факт Тризим Кап': region_combinations['Факт Тризим Кап'].sum(),
    '% Тризим Кап': (region_combinations['Факт Тризим Кап'].sum() / region_combinations['Тризим Кап'].sum()) * 100,
    'Энтеросгель': region_combinations['Энтеросгель'].sum(),
    'Факт Энтеросгель': region_combinations['Факт Энтеросгель'].sum(),
    '% Энтеросгель': (region_combinations['Факт Энтеросгель'].sum() / region_combinations['Энтеросгель'].sum()) * 100
}, name='TOTAL')

region_combinations = pd.concat([region_combinations, total_row.to_frame().transpose()], ignore_index=True)
region_combinations = region_combinations[['Регион', 'TO', 'Факт ТО', '% TO', 'Охват', 'Риномакс', 'Факт Риномакс',
                                           '% Риномакс', 'Форсил', 'Факт Форсил', '% Форсил', 'Тризим Таб',
                                           'Факт Тризим Таб', 'Альцетро', 'Факт Альцетро',
                                           '% Тризим Таб', 'Тризим Кап', 'Факт Тризим Кап', '% Тризим Кап',
                                           'Энтеросгель', 'Факт Энтеросгель', '% Энтеросгель']]


In [ ]:
region_combinations.to_excel('Salom.xlsx', index=False)

In [ ]:
float_cols = [['TO', 'Факт ТО', 'Охват',
               'Альцетро', 'Факт Альцетро',
               'Форсил', 'Факт Форсил',
               'Риномакс', 'Факт Риномакс',
               'Тризим Таб', 'Факт Тризим Таб',
               'Тризим Кап', 'Факт Тризим Кап',
               'Энтеросгель', 'Факт Энтеросгель']]
region_combinations[float_cols] = region_combinations[float_cols].astype('float64')

In [ ]:
formatter(region_combinations, 'Salom.xlsx')